# Lesson 4: Technical Debt

#### Setup
Set the ~~MakerSuite~~ Gemini API key with the provided helper function.

In [1]:
from utils import get_api_key

# PaLM legacy
## import google.generativeai as palm
## palm.configure(api_key=get_api_key())

# Gemini API
import os
import google.generativeai as genai
from google.api_core import client_options as client_options_lib

genai.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

#### Pick the model that generates text

```Python
# Legacy models shown in the video
models = [m for m in genai.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
# Model Bison set as legacy model in 2024
model_bison
```

In [2]:
# Set the model to connect to the Gemini API
model_flash = genai.GenerativeModel(model_name='gemini-2.0-flash')

#### Helper function to call the PaLM API

```Python
from google.api_core import retry
@retry.Retry()
def generate_text(prompt, 
                  model=model_bison, 
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)
```

### Helper function to call the Gemini API

In [3]:
def generate_text(prompt,
                  model=model_flash,
                  temperature=0.0):
    return model_flash.generate_content(prompt,
                                  generation_config={'temperature':temperature})

### Ask an LLM to explain a complex code base

In [10]:
#@title Complex Code Block
# Note: Taken from https://github.com/lmoroney/odmlbook/blob/63c0825094b2f44efc5c4d3226425a51990e73d6/BookSource/Chapter08/ios/cats_vs_dogs/CatVsDogClassifierSample/ModelDataHandler/ModelDataHandler.swift
CODE_BLOCK = """
// Copyright 2019 The TensorFlow Authors. All Rights Reserved.
//
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
//    http://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

import CoreImage
import TensorFlowLite
import UIKit


/// An inference from invoking the `Interpreter`.
struct Inference {
  let confidence: Float
  let label: String
}

/// Information about a model file or labels file.
typealias FileInfo = (name: String, extension: String)

/// Information about the MobileNet model.
enum MobileNet {
  static let modelInfo: FileInfo = (name: "converted_model", extension: "tflite")
}

/// This class handles all data preprocessing and makes calls to run inference on a given frame
/// by invoking the `Interpreter`. It then formats the inferences obtained and returns the top N
/// results for a successful inference.
class ModelDataHandler {

  // MARK: - Public Properties

  /// The current thread count used by the TensorFlow Lite Interpreter.
  let threadCount: Int

  let resultCount = 1

  // MARK: - Model Parameters

  let batchSize = 1
  let inputChannels = 3
  let inputWidth = 224
  let inputHeight = 224

  // MARK: - Private Properties

  /// List of labels from the given labels file.
  private var labels: [String] = ["Cat", "Dog"]

  /// TensorFlow Lite `Interpreter` object for performing inference on a given model.
  private var interpreter: Interpreter

  /// Information about the alpha component in RGBA data.
  private let alphaComponent = (baseOffset: 4, moduloRemainder: 3)

  // MARK: - Initialization

  /// A failable initializer for `ModelDataHandler`. A new instance is created if the model and
  /// labels files are successfully loaded from the app's main bundle. Default `threadCount` is 1.
  init?(modelFileInfo: FileInfo, threadCount: Int = 1) {
    let modelFilename = modelFileInfo.name

    // Construct the path to the model file.
    guard let modelPath = Bundle.main.path(
      forResource: modelFilename,
      ofType: modelFileInfo.extension
      ) else {
        print("Failed to load the model file with name: \(modelFilename).")
        return nil
    }

    // Specify the options for the `Interpreter`.
    self.threadCount = threadCount
    var options = InterpreterOptions()
    options.threadCount = threadCount
    do {
      // Create the `Interpreter`.
      interpreter = try Interpreter(modelPath: modelPath, options: options)
    } catch let error {
      print("Failed to create the interpreter with error: \(error.localizedDescription)")
      return nil
    }

  }

  // MARK: - Public Methods

  /// Performs image preprocessing, invokes the `Interpreter`, and process the inference results.
  func runModel(onFrame pixelBuffer: CVPixelBuffer) -> [Inference]? {
    let sourcePixelFormat = CVPixelBufferGetPixelFormatType(pixelBuffer)
    assert(sourcePixelFormat == kCVPixelFormatType_32ARGB ||
      sourcePixelFormat == kCVPixelFormatType_32BGRA ||
      sourcePixelFormat == kCVPixelFormatType_32RGBA)


    let imageChannels = 4
    assert(imageChannels >= inputChannels)

    // Crops the image to the biggest square in the center and scales it down to model dimensions.
    let scaledSize = CGSize(width: inputWidth, height: inputHeight)
    guard let thumbnailPixelBuffer = pixelBuffer.centerThumbnail(ofSize: scaledSize) else {
      return nil
    }

    let outputTensor: Tensor
    do {
      // Allocate memory for the model's input `Tensor`s.
      try interpreter.allocateTensors()

      // Remove the alpha component from the image buffer to get the RGB data.
      guard let rgbData = rgbDataFromBuffer(
        thumbnailPixelBuffer,
        byteCount: batchSize * inputWidth * inputHeight * inputChannels
        ) else {
          print("Failed to convert the image buffer to RGB data.")
          return nil
      }

      // Copy the RGB data to the input `Tensor`.
      try interpreter.copy(rgbData, toInputAt: 0)

      // Run inference by invoking the `Interpreter`.
      try interpreter.invoke()

      // Get the output `Tensor` to process the inference results.
      outputTensor = try interpreter.output(at: 0)
    } catch let error {
      print("Failed to invoke the interpreter with error: \(error.localizedDescription)")
      return nil
    }

    let results = [Float32](unsafeData: outputTensor.data) ?? []

    // Process the results.
    let topNInferences = getTopN(results: results)

    // Return the inference time and inference results.
    return topNInferences
  }

  // MARK: - Private Methods

  /// Returns the top N inference results sorted in descending order.
  private func getTopN(results: [Float]) -> [Inference] {
    // Create a zipped array of tuples [(labelIndex: Int, confidence: Float)].
    let zippedResults = zip(labels.indices, results)

    // Sort the zipped results by confidence value in descending order.
    let sortedResults = zippedResults.sorted { $0.1 > $1.1 }.prefix(resultCount)

    // Return the `Inference` results.
    return sortedResults.map { result in Inference(confidence: result.1, label: labels[result.0]) }
  }

  /// Loads the labels from the labels file and stores them in the `labels` property.
  private func loadLabels(fileInfo: FileInfo) {
    let filename = fileInfo.name
    let fileExtension = fileInfo.extension
    guard let fileURL = Bundle.main.url(forResource: filename, withExtension: fileExtension) else {
      fatalError("Labels file not found in bundle. Please add a labels file with name " +
        "\(filename).\(fileExtension) and try again.")
    }
    do {
      let contents = try String(contentsOf: fileURL, encoding: .utf8)
      labels = contents.components(separatedBy: .newlines)
    } catch {
      fatalError("Labels file named \(filename).\(fileExtension) cannot be read. Please add a " +
        "valid labels file and try again.")
    }
  }

  /// Returns the RGB data representation of the given image buffer with the specified `byteCount`.
  ///
  /// - Parameters
  ///   - buffer: The pixel buffer to convert to RGB data.
  ///   - byteCount: The expected byte count for the RGB data calculated using the values that the
  ///       model was trained on: `batchSize * imageWidth * imageHeight * componentsCount`.
  ///   - isModelQuantized: Whether the model is quantized (i.e. fixed point values rather than
  ///       floating point values).
  /// - Returns: The RGB data representation of the image buffer or `nil` if the buffer could not be
  ///     converted.
  private func rgbDataFromBuffer(
    _ buffer: CVPixelBuffer,
    byteCount: Int
    ) -> Data? {
    CVPixelBufferLockBaseAddress(buffer, .readOnly)
    defer { CVPixelBufferUnlockBaseAddress(buffer, .readOnly) }
    guard let mutableRawPointer = CVPixelBufferGetBaseAddress(buffer) else {
      return nil
    }
    let count = CVPixelBufferGetDataSize(buffer)
    let bufferData = Data(bytesNoCopy: mutableRawPointer, count: count, deallocator: .none)
    var rgbBytes = [Float](repeating: 0, count: byteCount)
    var index = 0
    for component in bufferData.enumerated() {
      let offset = component.offset
      let isAlphaComponent = (offset % alphaComponent.baseOffset) == alphaComponent.moduloRemainder
      guard !isAlphaComponent else { continue }
      rgbBytes[index] = Float(component.element) / 255.0
      index += 1
    }

    return rgbBytes.withUnsafeBufferPointer(Data.init)

  }
}

// MARK: - Extensions

extension Data {
  /// Creates a new buffer by copying the buffer pointer of the given array.
  ///
  /// - Warning: The given array's element type `T` must be trivial in that it can be copied bit
  ///     for bit with no indirection or reference-counting operations; otherwise, reinterpreting
  ///     data from the resulting buffer has undefined behavior.
  /// - Parameter array: An array with elements of type `T`.
  init<T>(copyingBufferOf array: [T]) {
    self = array.withUnsafeBufferPointer(Data.init)
  }
}

extension Array {
  /// Creates a new array from the bytes of the given unsafe data.
  ///
  /// - Warning: The array's `Element` type must be trivial in that it can be copied bit for bit
  ///     with no indirection or reference-counting operations; otherwise, copying the raw bytes in
  ///     the `unsafeData`'s buffer to a new array returns an unsafe copy.
  /// - Note: Returns `nil` if `unsafeData.count` is not a multiple of
  ///     `MemoryLayout<Element>.stride`.
  /// - Parameter unsafeData: The data containing the bytes to turn into an array.
  init?(unsafeData: Data) {

    guard unsafeData.count % MemoryLayout<Element>.stride == 0 else { return nil }
    #if swift(>=5.0)
    self = unsafeData.withUnsafeBytes { .init($0.bindMemory(to: Element.self)) }
    #else
    self = unsafeData.withUnsafeBytes {
      .init(UnsafeBufferPointer<Element>(
        start: $0,
        count: unsafeData.count / MemoryLayout<Element>.stride
      ))
    }
    #endif  // swift(>=5.0)
  }
}
"""

In [5]:
prompt_template = """
Can you please explain how this code works?

{question}

Use a lot of detail and make it as clear as possible.
"""

In [6]:
completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)

# Gemini API
print(completion.text)

# PaLM legacy
## print(completion.result)

Okay, let's break down this Swift code, which is designed to perform image classification using a TensorFlow Lite model.  It's a common pattern for mobile applications that need to analyze images quickly and efficiently.

**Overall Purpose**

The code defines a `ModelDataHandler` class that encapsulates the logic for:

1.  **Loading a TensorFlow Lite model:**  It reads the model file from the app's bundle.
2.  **Preprocessing images:**  It takes a `CVPixelBuffer` (a common image format in iOS) as input, resizes it, and converts it into a format suitable for the TensorFlow Lite model.
3.  **Running inference:**  It uses the TensorFlow Lite `Interpreter` to execute the model on the preprocessed image data.
4.  **Postprocessing results:**  It extracts the model's output, finds the most likely classification labels, and returns them as `Inference` objects.

**Code Breakdown**

Let's go through the code section by section:

**1. Copyright and Imports**

```swift
// Copyright 2019 The Tensor

#### Try it out on your own code!
- Try inputting some code into the `CODE_BLOCK` variable.

In [7]:
CODE_BLOCK2 = """ 
# replace this with your own code
def foo(a):
  b = a + 1
  return 2*b
"""

prompt_template = """
Can you please explain how this code works?

{question}

Use a lot of detail and make it as clear as possible.
"""

completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK2)
)
print(completion.text)

Okay, let's break down this Python code snippet step-by-step.

**Code:**

```python
def foo(a):
  b = a + 1
  return 2*b
```

**Explanation:**

1.  **`def foo(a):`**:
    *   `def` is a keyword in Python that signifies the beginning of a function definition.  A function is a reusable block of code that performs a specific task.
    *   `foo` is the name we've given to this function.  You can choose almost any valid name for your functions (following Python's naming rules).  It's good practice to choose a name that describes what the function does.
    *   `(a)`:  This part defines the *parameters* of the function.  In this case, the function `foo` takes one parameter, which we've named `a`.  A parameter is a variable that the function expects to receive as input when it's called.  Think of it as a placeholder for a value that will be provided later.

2.  **`b = a + 1`**:
    *   This line is inside the function's *body*.  The body contains the instructions that the function will execut

### Ask an LLM to document a complex code base

In [8]:
prompt_template = """
Please write technical documentation for this code and \n
make it easy for a non swift developer to understand:

{question}

Output the results in markdown
"""

In [11]:
completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)
print(completion.text)

```markdown
## Technical Documentation for Image Classification Model Handler (Swift)

This document describes the `ModelDataHandler` class, a Swift class designed to perform image classification using a TensorFlow Lite model. It's intended for developers who may not be familiar with Swift but need to understand how this code works.

**Purpose:**

The `ModelDataHandler` class is responsible for:

1.  **Loading a TensorFlow Lite model:**  It loads a pre-trained model from your application's resources.
2.  **Preprocessing images:** It takes an image (represented as a `CVPixelBuffer`) and prepares it for the model by resizing and converting it to the correct format.
3.  **Running inference:** It feeds the preprocessed image to the TensorFlow Lite interpreter and gets the model's predictions.
4.  **Post-processing results:** It interprets the model's output and returns the top prediction(s) with their associated confidence scores.

**Key Concepts:**

*   **TensorFlow Lite (TFLite):** A lig

### Try it out on your own code!
- Notice that we've modified the prompt slightly to refer to Python instead of Swift.

In [12]:
CODE_BLOCK2 = """ 
# replace this with your own code
def foo(a):
  b = a + 1
  return 2*b
"""

prompt_template = """
Please write technical documentation for this code and \n
make it easy for a non Python developer to understand:

{question}

Output the results in markdown
"""

completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK2)
)
print(completion.text)

```markdown
## Technical Documentation for the `foo` Function

This document describes the functionality of a simple function named `foo`.  It's written to be understandable even if you don't know Python.

**1. Purpose**

The `foo` function takes a single numerical input, performs a calculation, and returns a numerical result.  Think of it like a mini-calculator that does a specific set of operations.

**2. Input (Parameters)**

*   **Name:** `a`
*   **Type:**  A number (integer or decimal).  The function is designed to work with numerical values.
*   **Description:**  This is the initial number that the function will use in its calculations.  It's the starting point.

**3. Process (How it Works)**

The function performs the following steps:

1.  **Add 1:** It takes the input number `a` and adds 1 to it.  The result of this addition is stored in a temporary variable called `b`.  So, `b = a + 1`.

2.  **Multiply by 2:** It then takes the value of `b` (which is `a + 1`) and multiplies it